In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()  # Enable progress_apply with progress bar in Jupyter environments
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.matutils import Sparse2Corpus
from gensim import models
from scipy.sparse import csr_matrix
import missingno as msno
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm


In [ ]:
df = pd.read_excel('/Users/ruimaciel/Desktop/Barcelona/Master_Thesis/ECB_Perceived_Cacophony/combined_updated.xlsx')
df.head(20)

In [ ]:
#import plotly.io as pio
#pio.renderers.default = 'notebook'


# PreProcess Scraped Text

In [ ]:
df_scraped = pd.read_excel('/Users/ruimaciel/Desktop/Barcelona/Master_Thesis/ECB_Perceived_Cacophony/combined_updated.xlsx')
df_scraped.head(20)

In [ ]:
df_scraped.shape

In [ ]:
# Select rows where both 'Manual.summary' and 'Translated.text' are null or empty
null_rows = df[(df['Manual.summary'].isnull() | df['Manual.summary'].eq('')) & 
               (df['Translated.text'].isnull() | df['Translated.text'].eq(''))]

# Print the selected rows
print(null_rows)


In [ ]:
# Select rows where both 'Manual.summary' and 'Translated.text' are null or empty or contain '-'
null_rows = df_scraped[(df_scraped['Manual.summary'].isnull() | df_scraped['Manual.summary'].eq('') | df_scraped['Manual.summary'].eq('-')) & 
                       (df_scraped['Translated.text'].isnull() | df_scraped['Translated.text'].eq(''))]

# Count the number of rows where both 'Manual.summary' and 'Translated.text' are null or empty or contain '-'
count_both_missing = null_rows.shape[0]

# Calculate the total number of rows in the DataFrame
total_rows = df_scraped.shape[0]

# Calculate the percentage of rows where both 'Manual.summary' and 'Translated.text' are null or empty or contain '-'
percentage_both_missing = (count_both_missing / total_rows) * 100

# Print the number of rows and the percentage of total rows where both 'Manual.summary' and 'Translated.text' are null or empty or contain '-'
print(f"Number of rows where both 'Manual.summary' and 'Translated.text' are null, empty, or contain '-': {count_both_missing}")
print(f"Percentage of total rows where both 'Manual.summary' and 'Translated.text' are null, empty, or contain '-': {percentage_both_missing:.2f}%")

# Print the selected rows
print("\nRows where both 'Manual.summary' and 'Translated.text' are null, empty, or contain '-':")
print(null_rows)


### Preprocess Manual Summary and Scraped Text

In [ ]:
# Download necessary NLTK resources if not already available
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Prepare the stopwords set once to avoid repeated loading
initial_stop_words = stopwords.words('english')
custom_stopwords = {
    'npagina', 'nfoglio', 'nquotidiano', 'ndi', 'nla', 'ndie', 'nder', 'ne', 'nlos', 'nde', 
    'na', 'nto', 'np', 'nel', 'nque', 'nen', 'ndel', 'cm'
}
stop_words = set(initial_stop_words).union(custom_stopwords)

def clean_text(text):
    """
    Perform initial cleaning of the text by removing URLs, numbers,
    non-alphabetic characters, and converting to lowercase.
    """
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\b\d+\b', '', text)  # Remove all standalone numbers
    text = re.sub(r'\bn[a-z]{2,}\b', '', text)  # Aggressively remove words starting with 'n' followed by at least two letters
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabet characters
    return text.lower()

def tokenize(text, mode):
    """
    Tokenizes the text based on the given mode.
    Mode 0: Convert text to lowercase.
    Mode 1: Apply stemming.
    Mode 2: Apply lemmatization.
    """
    text = clean_text(text)  # Apply enhanced cleaning
    words = word_tokenize(text)
    if mode == 1:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words if word not in stop_words]
    elif mode == 2:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    else:
        words = [word for word in words if word not in stop_words]

    return ' '.join(words)

def preprocess_texts(data_frame, column_name, mode):
    """
    Apply text preprocessing to a specified column in a DataFrame.
    """
    # Fill NaN values with empty strings before processing
    data_frame[column_name] = data_frame[column_name].fillna('')
    # Use tqdm to display progress while applying preprocessing
    tqdm.pandas(desc="Processing Texts")
    return data_frame[column_name].astype(str).progress_apply(lambda row: tokenize(row, mode))

In [ ]:
import numpy as np
mod = 2  # Set the preprocessing mode to 2 for lemmatizing

# Reset the index of the DataFrame to avoid duplicate labels
df_scraped.reset_index(drop=True, inplace=True)

# Determine which column to preprocess for translated text
df_scraped['text_to_preprocess'] = np.where(df_scraped['Translated.text'].fillna('').str.strip() == '',
                                            df_scraped['Manual.summary'], 
                                            df_scraped['Translated.text'])

# Apply preprocessing to the determined text
df_scraped['translated_text_preproc'] = preprocess_texts(df_scraped, 'text_to_preprocess', mod)

# Apply preprocessing to the Manual.summary column
df_scraped['manual_summary_preproc'] = preprocess_texts(df_scraped, 'Manual.summary', mod)

# Remove the auxiliary column if no longer needed
df_scraped.drop(columns=['text_to_preprocess'], inplace=True)

# Display the result to verify
display(df_scraped[['Source', 'Manual.summary', 'Translated.text', 'Language', 'translated_text_preproc', 'manual_summary_preproc']].head(20))

# ECB Governors Checking

In [ ]:
ECB_df = pd.read_csv('ecb_governing_council.csv')
ECB_df

In [ ]:
# Function to generate name variations
def generate_name_variations(full_name, first_name, surname):
    variations = [
        full_name,
        first_name,
        surname,
        f"{first_name} {surname}",
        f"{surname} {first_name}",
        full_name.replace(" ", ""),
        first_name.replace(" ", ""),
        surname.replace(" ", ""),
    ]
    return list(set(variations))  # Ensure unique variations

# Function to count occurrences of names
def count_occurrences(text, names):
    count = 0
    for name in names:
        count += len(re.findall(rf'\b{name}\b', text, flags=re.IGNORECASE))
    return count

# Extract names and countries from ECB_df
full_names = ECB_df['Full Name'].tolist()
first_names = ECB_df['First Name'].tolist()
surnames = ECB_df['Surname'].tolist()
countries = ECB_df['Country'].tolist()  # Update variable name to 'countries'

# Create a list to store the results
results = []

# Generate name variations for each governor
name_variations_list = [
    generate_name_variations(row['Full Name'], row['First Name'], row['Surname'])
    for index, row in ECB_df.iterrows()
]

# Iterate through the governors and count occurrences in the preprocessed columns
for index, row in ECB_df.iterrows():
    full_name = row['Full Name']
    first_name = row['First Name']
    surname = row['Surname']
    country = row['Country']  # Include the country

    name_variations = generate_name_variations(full_name, first_name, surname)

    full_name_count_translated = df_scraped['translated_text_preproc'].apply(lambda x: count_occurrences(x, name_variations)).sum()
    first_name_count_translated = df_scraped['translated_text_preproc'].apply(lambda x: count_occurrences(x, [first_name])).sum()
    surname_count_translated = df_scraped['translated_text_preproc'].apply(lambda x: count_occurrences(x, [surname])).sum()

    full_name_count_manual = df_scraped['manual_summary_preproc'].apply(lambda x: count_occurrences(x, name_variations)).sum()
    first_name_count_manual = df_scraped['manual_summary_preproc'].apply(lambda x: count_occurrences(x, [first_name])).sum()
    surname_count_manual = df_scraped['manual_summary_preproc'].apply(lambda x: count_occurrences(x, [surname])).sum()

    results.append({
        'Governor': full_name,
        'Country': country,  # Include the country in the results
        'Full Name Count (Scraped Text)': full_name_count_translated,
        'First Name Count (Scraped Text)': first_name_count_translated,
        'Surname Count (Scraped Text)': surname_count_translated,
        'Full Name Count (Manual Summary)': full_name_count_manual,
        'First Name Count (Manual Summary)': first_name_count_manual,
        'Surname Count (Manual Summary)': surname_count_manual
    })

# Convert the results list to a DataFrame
count_df = pd.DataFrame(results)

# Display the result to verify
count_df.head(30)


In [ ]:
# Function to generate name variations
def generate_name_variations(full_name, first_name, surname):
    variations = [
        full_name,
        first_name,
        surname,
        f"{first_name} {surname}",
        f"{surname} {first_name}",
        full_name.replace(" ", ""),
        first_name.replace(" ", ""),
        surname.replace(" ", ""),
    ]
    return list(set(variations))  # Ensure unique variations

# Function to count occurrences of names
def count_occurrences(text, names):
    count = 0
    for name in names:
        count += len(re.findall(rf'\b{name}\b', text, flags=re.IGNORECASE))
    return count

# Extract names and countries from ECB_df
full_names = ECB_df['Full Name'].tolist()
first_names = ECB_df['First Name'].tolist()
surnames = ECB_df['Surname'].tolist()
countries = ECB_df['Country'].tolist()  # Update variable name to 'countries'

# Create a list to store the results
results = []

# Generate name variations for each governor
name_variations_list = [
    generate_name_variations(row['Full Name'], row['First Name'], row['Surname'])
    for index, row in ECB_df.iterrows()
]

# Iterate through the governors and count occurrences in the preprocessed columns
for index, row in ECB_df.iterrows():
    full_name = row['Full Name']
    first_name = row['First Name']
    surname = row['Surname']
    country = row['Country']  # Include the country

    name_variations = generate_name_variations(full_name, first_name, surname)

    full_name_count_translated = df_scraped['translated_text_preproc'].apply(lambda x: count_occurrences(x, name_variations)).sum()
    first_name_count_translated = df_scraped['translated_text_preproc'].apply(lambda x: count_occurrences(x, [first_name])).sum()
    surname_count_translated = df_scraped['translated_text_preproc'].apply(lambda x: count_occurrences(x, [surname])).sum()

    full_name_count_manual = df_scraped['manual_summary_preproc'].apply(lambda x: count_occurrences(x, name_variations)).sum()
    first_name_count_manual = df_scraped['manual_summary_preproc'].apply(lambda x: count_occurrences(x, [first_name])).sum()
    surname_count_manual = df_scraped['manual_summary_preproc'].apply(lambda x: count_occurrences(x, [surname])).sum()

    results.append({
        'Governor': full_name,
        'Country': country,  # Include the country in the results
        'Full Name Count (Scraped Text)': full_name_count_translated,
        'First Name Count (Scraped Text)': first_name_count_translated,
        'Surname Count (Scraped Text)': surname_count_translated,
        'Full Name Count (Manual Summary)': full_name_count_manual,
        'First Name Count (Manual Summary)': first_name_count_manual,
        'Surname Count (Manual Summary)': surname_count_manual
    })

# Convert the results list to a DataFrame
count_df = pd.DataFrame(results)

# Display the result to verify
count_df.head(30)


In [ ]:
import pandas as pd
import re

df_scraped['Name_of_Speaker'] = None  # Initialize the column

def check_and_return_full_name(text, name_variations, full_name):
    # First, check for the specific case of Martins Kazaks
    martins_kazaks_variations = ['mrti kazks', 'mrti', 'kazks']
    for mk_name in martins_kazaks_variations:
        if re.search(rf'\b{mk_name}\b', text, flags=re.IGNORECASE):
            return 'Martins Kazaks'
    # Then check other name variations
    for name in name_variations:
        if re.search(rf'\b{name}\b', text, flags=re.IGNORECASE):
            return full_name
    return None

# Assume generate_name_variations is a function that takes full name, first name, and surname and returns a list of name variations
# Iterate through each row in ECB_df to check matches in df_scraped
for index, row in ECB_df.iterrows():
    full_name = row['Full Name']
    name_variations = generate_name_variations(full_name, row['First Name'], row['Surname'])

    # Update df_scraped 'Name_of_Speaker' column where matches are found
    df_scraped['Name_of_Speaker'] = df_scraped.apply(
        lambda x: check_and_return_full_name(x['translated_text_preproc'], name_variations, full_name)
        if x['Name_of_Speaker'] is None else x['Name_of_Speaker'], axis=1)

    df_scraped['Name_of_Speaker'] = df_scraped.apply(
        lambda x: check_and_return_full_name(x['manual_summary_preproc'], name_variations, full_name)
        if x['Name_of_Speaker'] is None else x['Name_of_Speaker'], axis=1)

# Print updated DataFrame
print(df_scraped.head())


In [ ]:
# Assuming df_scraped is already loaded with data

# Count occurrences of each unique value in 'Name_of_Speaker'
name_counts = df_scraped['Name_of_Speaker'].value_counts()
print(name_counts)


# Plotting the counts
plt.figure(figsize=(10, 8))  # Set the figure size for better readability
name_counts.plot(kind='bar', color='skyblue')  # Create a bar plot
plt.title('Count of Each Speaker in Data')  # Title of the plot
plt.xlabel('Name of Speaker')  # Label for the x-axis
plt.ylabel('Counts')  # Label for the y-axis
plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better readability
plt.tight_layout()  # Adjust subplots to give some padding
plt.show()  # Display the plot

In [ ]:
#df_scraped.to_csv('df_scraped_with_names.csv', index=False)

# Checking the non-check

In [ ]:
df_scraped_2=pd.read_csv('df_scraped_with_names.csv')
df_scraped_2.head(20)

In [ ]:
# Print the rows which Manual.Summary is Nan or it has less than 10 characters
print(df_scraped_2[(df_scraped_2['manual_summary_preproc'].isnull()) & (df_scraped_2['Name_of_Speaker'].isnull())])
# Count the rows which Manual.Summary is Nan or it has less than 10 characters
print(df_scraped_2[(df_scraped_2['manual_summary_preproc'].isnull()) & (df_scraped_2['Name_of_Speaker'].isnull())].shape[0])


In [ ]:
#Create a dataframe only with the manual_summary_preproc is null
df_scraped_null = df_scraped_2[(df_scraped_2['manual_summary_preproc'].isnull()) & (df_scraped_2['Name_of_Speaker'].isnull())]
df_scraped_null.head(20)


In [ ]:
print(df_scraped_null.shape[0])

In [ ]:
# Print the rows where both 'manual_summary_preproc' and 'Name_of_Speaker' are NaN
print(df_scraped_2[(df_scraped_2['manual_summary_preproc'].isnull()) & (df_scraped_2['Name_of_Speaker'].isnull())])

# Print the total number of rows before removing any
print("Total rows before removing any:", df_scraped_2.shape[0])

# Remove rows where 'manual_summary_preproc' is null or 'Name_of_Speaker' is null
df_scraped_2 = df_scraped_2[~(df_scraped_2['manual_summary_preproc'].isnull() & df_scraped_2['Name_of_Speaker'].isnull())]

# Print the total number of rows after removing rows where 'manual_summary_preproc' or 'Name_of_Speaker' is null
print("Total rows after removing rows with any NaNs in 'manual_summary_preproc' or 'Name_of_Speaker':", df_scraped_2.shape[0])


In [ ]:
#Check the percentage of null_rows in the 'Name_of_Speaker' column
null_rows = df_scraped[df_scraped['Name_of_Speaker'].isnull()]
count_both_missing = null_rows.shape[0]
total_rows = df_scraped.shape[0]
percentage_both_missing_1 = (count_both_missing / total_rows) * 100

print(f"Number of rows where 'Name_of_Speaker' is null in df_shaped: {count_both_missing}")
print (f"Percentage of total rows where 'Name_of_Speaker' is null in df_shaped: {percentage_both_missing_1:.2f}%")

#Check the percentage of null_rows in the 'Name_of_Speaker' column
null_rows = df_scraped_2[df_scraped_2['Name_of_Speaker'].isnull()]
count_both_missing = null_rows.shape[0]
total_rows = df_scraped_2.shape[0]
percentage_both_missing_2 = (count_both_missing / total_rows) * 100

print(f"Number of rows where 'Name_of_Speaker' is null in df_shaped_2: {count_both_missing}")
print (f"Percentage of total rows where 'Name_of_Speaker' is null in df_shaped_2: {percentage_both_missing_2:.2f}%")

In [ ]:
# Print the percentage of rows where 'Name_of_Speaker' is null
print(f"Percentage of total rows where 'Name_of_Speaker' is null: {percentage_both_missing:.2f}%")
print(f'{count_both_missing} rows where Name_of_Speaker is null')
# Print the total number of rows
print(f"Total number of rows: {total_rows}")



# Data Quality Check and Duplicates

In [ ]:
# Set display options for wide windows
pd.set_option('display.max_colwidth', None)  # Use None to show full content without truncation
pd.set_option('display.width', 1000)  # Adjust the number to fit your display or preferences


In [ ]:
df_scraped_2.shape[0]

In [ ]:
# Step 1: Count the number of rows without a Name_of_Speaker
missing_names_count = df_scraped_2['Name_of_Speaker'].isna().sum()
print(f"Number of rows without Name_of_Speaker: {missing_names_count}")

# Step 2: Remove rows where Name_of_Speaker is missing
df_cleaned = df_scraped_2.dropna(subset=['Name_of_Speaker'])

# Optionally print the shape to confirm rows are removed
print(f"New DataFrame shape after removing rows without Name_of_Speaker: {df_cleaned.shape}")


In [ ]:
# Step 1: Count the number of rows without a Manual.summary
missing_summaries_count = df_scraped_2['Manual.summary'].isna().sum()
print(f"Number of rows without Manual.summary: {missing_summaries_count}")

# Step 2: Remove rows where Manual.summary is missing
df_cleaned = df_scraped_2.dropna(subset=['Manual.summary'])

# Optionally print the shape to confirm rows are removed
print(f"New DataFrame shape after removing rows without Manual.summary: {df_cleaned.shape}")

In [ ]:
# List of columns to check for duplicates
columns_to_check = [
    'Date', 'Source', 'Headline', 'Translated.headline', 'Manual.summary', 
    'Original.article.url', 'Translated.text', 'Media.type', 'Speaker', 
    'Reach', 'Language', 'Country.Code', 'Sitename', 'Tags'
]

# Identifying duplicates based on the list of columns
duplicates = df_scraped_2[df_scraped_2.duplicated(columns_to_check, keep=False)]

# Sorting the duplicates to group identical entries together
duplicates_sorted = duplicates.sort_values(by=columns_to_check)

print(f"Number of duplicate entries: {duplicates_sorted.shape[0]}")
duplicates_sorted

In [ ]:
# Remove duplicates based on the list of columns
df_cleaned = df_scraped_2.drop_duplicates(subset=columns_to_check, keep='first')

# Print the shape to confirm rows are removed
print(f"DataFrame shape after removing duplicates: {df_cleaned.shape}")

In [ ]:
# Calculate the number of manual summaries with less than 30 characters
count_less_than_30 = (df_scraped_2['Manual.summary'].str.len() <30).sum()

print(f"Number of manual summaries with less than 30 characters: {count_less_than_30}")

# Filter and print only the 'Manual.summary' column where summaries have less than 30 characters
summaries_less_than_30 = df_scraped_2.loc[df_scraped_2['Manual.summary'].str.len() < 30, 'Manual.summary']
print(summaries_less_than_30)

In [ ]:
# Filter to remove rows where the 'Manual.summary' column has summaries less than 30 characters
df_cleaned = df_scraped_2[df_scraped_2['Manual.summary'].str.len() >= 30]

# Optionally print the cleaned DataFrame to confirm rows are removed
print(f"DataFrame shape after removing short summaries: {df_cleaned.shape}")
print(df_cleaned)

# Checking that everythins is similar afterwards

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Count the occurrences for each language
language_counts = df_cleaned['Language'].value_counts().reset_index()
language_counts.columns = ['Language', 'Count']

# Count the occurrences for each country code
country_code_counts = df['Country.Code'].value_counts().reset_index()
country_code_counts.columns = ['Country Code', 'Count']

# Create a figure with one row and two columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Article Counts by Language', 'Article Counts by Country Code'))

# Plotting the 'Language' bar plot with Plotly in the first column
fig.add_trace(
    go.Bar(x=language_counts['Language'], y=language_counts['Count'], 
           marker=dict(color=px.colors.sequential.Blues[-2])),
    row=1, col=1
)

# Plotting the 'Country Code' bar plot with Plotly in the second column
fig.add_trace(
    go.Bar(x=country_code_counts['Country Code'], y=country_code_counts['Count'], 
           marker=dict(color=px.colors.sequential.Reds[-2])),
    row=1, col=2
)

# Update xaxis properties for both plots for better readability
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)

# Update layout to adjust spacing and add some styling
fig.update_layout(
    height=500,  # Adjusted for better proportion in side-by-side layout
    width=1500,  # Adjust width to accommodate side-by-side layout
    showlegend=False,
    title_text="Article Counts by Language and Country Code",
    title_x=0.5  # Centering the title
)

fig.show()


In [ ]:
# Assuming df_scraped is already loaded with data

# Count occurrences of each unique value in 'Name_of_Speaker'
name_counts = df_cleaned['Name_of_Speaker'].value_counts()
print(name_counts)


# Plotting the counts
plt.figure(figsize=(10, 8))  # Set the figure size for better readability
name_counts.plot(kind='bar', color='skyblue')  # Create a bar plot
plt.title('Count of Each Speaker in Data')  # Title of the plot
plt.xlabel('Name of Speaker')  # Label for the x-axis
plt.ylabel('Counts')  # Label for the y-axis
plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better readability
plt.tight_layout()  # Adjust subplots to give some padding
plt.show()  # Display the plot

In [ ]:
# Count the occurrences of each media type
media_type_counts = df_cleaned['Media.type'].value_counts()

# Create a DataFrame from the media type counts
media_type_df = media_type_counts.reset_index()
media_type_df.columns = ['Media.type', 'Count']  # Rename columns for clarity

# Show the top 10 most frequent media types
print(media_type_df.head(10))

In [ ]:
# Count of articles per Sitename
site_article_counts = df_cleaned['Sitename'].value_counts()

site_article_counts.head(10)


In [ ]:
asdasdas

In [ ]:
#df_cleaned.to_csv('df_cleaned_final_01.csv', index=False)

# DOUBLE CHECK

In [ ]:
df_final=pd.read_csv('/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/df_cleaned_final_01.csv')

In [ ]:
#Check missing values on Name_of_Speaker
missing_names_count = df_final['Name_of_Speaker'].isna().sum()
print(f"Number of rows without Name_of_Speaker: {missing_names_count}")

In [ ]:
#Drop rows with missing values on Name_of_Speaker
df_final = df_final.dropna(subset=['Name_of_Speaker'])

#Print of shape of the dataframe after removing missing values on Name_of_Speaker
print(f"New DataFrame shape after removing rows without Name_of_Speaker: {df_final.shape}")

In [ ]:
#Check missing values on Manual.summary
missing_summaries_count = df_final['Manual.summary'].isna().sum()
print(f"Number of rows without Manual.summary: {missing_summaries_count}")

# List of columns to check for duplicates
columns_to_check = [
    'Date', 'Source', 'Headline', 'Translated.headline', 'Manual.summary', 
    'Original.article.url', 'Translated.text', 'Media.type', 'Speaker', 
    'Reach', 'Language', 'Country.Code', 'Sitename', 'Tags'
]

# Identifying duplicates based on the list of columns
duplicates = df_final[df_final.duplicated(columns_to_check, keep=False)]

# Sorting the duplicates to group identical entries together
duplicates_sorted = duplicates.sort_values(by=columns_to_check)

print(f"Number of duplicate entries: {duplicates_sorted.shape[0]}")
duplicates_sorted

# Remove duplicates based on the list of columns
df_final = df_final.drop_duplicates(subset=columns_to_check, keep='first')
# Print the shape to confirm rows are removed
print(f"DataFrame shape after removing duplicates: {df_final.shape}")


In [ ]:
df_final.to_csv('/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/df_cleaned_final_01.csv', index=False)

### EDA Governors Analysis

In [ ]:
import plotly.graph_objects as go

#Create the bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=count_df['Governor'],
    y=count_df['Full Name Count (Scraped Text)'],
    name='Full Name Count (Scraped Text)'
))

fig.add_trace(go.Bar(
    x=count_df['Governor'],
    y=count_df['Full Name Count (Manual Summary)'],
    name='Full Name Count (Manual Summary)'
))

# Update the layout
fig.update_layout(
    title='Comparison of Full Name Counts',
    xaxis_title='Governor',
    yaxis_title='Count',
    barmode='group'
)

# Show the figure
fig.show()


In [ ]:

# Function to check for a match with variations
def has_match(text, name_variations):
    for name in name_variations:
        if re.search(rf'\b{name}\b', text, flags=re.IGNORECASE):
            return True
    return False

# Create a function to generate name variations
def generate_name_variations(full_name, first_name, surname):
    variations = [
        full_name,
        first_name,
        surname,
        f"{first_name} {surname}",
        f"{surname} {first_name}",
        full_name.replace(" ", ""),
        first_name.replace(" ", ""),
        surname.replace(" ", ""),
    ]
    return list(set(variations))  # Ensure unique variations

# List of full names and their variations
name_variations_list = [
    generate_name_variations(row['Full Name'], row['First Name'], row['Surname'])
    for index, row in ECB_df.iterrows()
]

# Check for matches in the preprocessed columns and count articles with at least one match
total_articles = len(df_scraped)
full_name_matches_translated = df_scraped['translated_text_preproc'].apply(
    lambda x: any(has_match(x, name_variations) for name_variations in name_variations_list)
).sum()
full_name_matches_manual = df_scraped['manual_summary_preproc'].apply(
    lambda x: any(has_match(x, name_variations) for name_variations in name_variations_list)
).sum()

# Calculate the proportion of articles with full name matches
full_name_proportion_translated = full_name_matches_translated / total_articles
full_name_proportion_manual = full_name_matches_manual / total_articles

print(f"Full Name Match Statistics:")
print(f"Total articles: {total_articles}")
print(f"Articles with full name match (Translated Text): {full_name_matches_translated} ({full_name_proportion_translated:.2%})")
print(f"Articles with full name match (Manual Summary): {full_name_matches_manual} ({full_name_proportion_manual:.2%})")

# Filter articles without full name matches in the manual summary column and ignore blank entries
articles_without_full_name_match_manual = df_scraped[df_scraped['manual_summary_preproc'].apply(
    lambda x: not any(has_match(x, name_variations) for name_variations in name_variations_list) and x.strip() != ''
)]

# Randomly select 10 examples of such articles
examples_without_full_name_match_manual = articles_without_full_name_match_manual.sample(n=10, random_state=1)

# Display the examples with full content in a more readable format
pd.set_option('display.max_colwidth', None)
print("\nExamples of articles without full name match (Manual Summary):")
for i, row in examples_without_full_name_match_manual.iterrows():
    print(f"Article {i+1}:\n{row['manual_summary_preproc']}\n{'='*80}\n")


In [ ]:
# Function to check for a match
def has_any_match(text, full_names, first_names, surnames):
    for name in full_names + first_names + surnames:
        if re.search(rf'\b{name}\b', text, flags=re.IGNORECASE):
            return True
    return False

# Lists of names
full_names = ECB_df['Full Name'].tolist()
first_names = ECB_df['First Name'].tolist()
surnames = ECB_df['Surname'].tolist()

# Check for matches in the preprocessed columns and count articles with at least one match
total_articles = len(df_scraped)
any_name_matches_translated = df_scraped['translated_text_preproc'].apply(
    lambda x: has_any_match(x, full_names, first_names, surnames)).sum()
any_name_matches_manual = df_scraped['manual_summary_preproc'].apply(
    lambda x: has_any_match(x, full_names, first_names, surnames)).sum()

# Calculate the proportion of articles with any name matches
any_name_proportion_translated = any_name_matches_translated / total_articles
any_name_proportion_manual = any_name_matches_manual / total_articles

print(f"\nFull Names + First Names + Surnames Match Statistics:")
print(f"Total articles: {total_articles}")
print(f"Articles with any name match (Translated Text): {any_name_matches_translated} ({any_name_proportion_translated:.2%})")
print(f"Articles with any name match (Manual Summary): {any_name_matches_manual} ({any_name_proportion_manual:.2%})")

# Filter articles without any name matches in the manual summary column and ignore blank entries
articles_without_any_name_match_manual = df_scraped[df_scraped['manual_summary_preproc'].apply(
    lambda x: not has_any_match(x, full_names, first_names, surnames) and x.strip() != ''
)]

# Randomly select 10 examples of such articles
examples_without_any_name_match_manual = articles_without_any_name_match_manual.sample(n=10, random_state=1)

# Display the examples with full content in a more readable format
pd.set_option('display.max_colwidth', None)
print("\nExamples of articles without any name match (Manual Summary):")
for i, row in examples_without_any_name_match_manual.iterrows():
    print(f"Article {i+1}:\n{row['manual_summary_preproc']}\n{'='*80}\n")


In [ ]:
ECB_df.columns

In [ ]:
# Save the DataFrame to a CSV file
articles_without_any_name_match_manual.to_csv('articles_without_any_name_match.csv', index=False)


### Spacy Analysis Hawkish & Dovish

In [ ]:
# Download the spaCy model (run this command only once)
#!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define expanded dovish and hawkish words
dovish_words = [
    "accommodative", "easing", "expansionary", "stimulus", "lower rates", "dovish", "supportive", "lowering rates",
    "quantitative easing", "QE", "liquidity", "easy money", "softening", "growth", "recovery", "reduce", "lowering",
    "decrease", "supportive measures", "boost", "economic support", "fiscal support", "monetary support", "rate cuts",
    "rate reduction", "interest rate cut", "low interest", "maintaining low rates", "deflationary", "deflation", "aid",
    "bailout", "funding", "cash flow", "intervention", "credit easing", "backstop"
]

hawkish_words = [
    "tightening", "contractionary", "restrictive", "hiking", "raising rates", "hawkish", "higher rates", "inflationary",
    "quantitative tightening", "QT", "interest rate hike", "hard money", "hardening", "inflation", "price stability",
    "curbing", "containing inflation", "reducing inflation", "anti-inflationary", "overheating", "fiscal discipline",
    "monetary tightening", "rate hikes", "rate increase", "interest rate rise", "high interest", "reducing stimulus",
    "economic restraint", "budget cuts", "austerity", "monetary discipline", "tapering", "withdrawal of stimulus",
    "debt control", "fiscal consolidation", "financial tightening"
]

# Custom stop words list
custom_stop_words = list(nlp.Defaults.stop_words.union({
    "rate", "ecb", "bank", "central", "euro", "said", "president", "increase", "inflation", "lagarde",
    "interest", "policy", "monetary", "market", "economy", "price", "point", "year", "high", "area", "christine", "mr", "de", "hike",
    "percent", "billion", "million", "trillion", "europe", "union", "currency", "eurosystem", "system", "committee",
    "board", "member", "members", "meeting", "conference", "discussion", "report", "statement", "notes", "speech",
    "publication", "data", "figures", "information", "details", "analysis", "comments", "comment", "commentary", 
    "remarks", "outlook", "forecast", "projections", "expectations", "view", "views", "opinion", "opinions", "perspective",
    "perspectives", "standpoint", "stance", "position", "positions", "policy", "policies", "approach", "approaches", "strategy",
    "strategies", "framework", "frameworks", "program", "programs", "measures", "measure", "tools", "tool", "instrument",
    "instruments", "mechanism", "mechanisms", "method", "methods", "procedure", "procedures", "process", "processes",
    "implementation", "practice", "practices", "execution", "operation", "operations", "activity", "activities", 
    "function", "functions", "role", "roles", "task", "tasks", "responsibility", "responsibilities", "duty", "duties",
    "obligation", "obligations", "commitment", "commitments", "engagement", "engagements", "undertaking", "undertakings", "according", "time", "according", "level", "council", "financial", "expected", "european"
}))

# Function to extract potential sentiment words using spaCy with progress bar
def extract_potential_words(texts, existing_words, label):
    potential_words = []
    for text in tqdm(texts, desc=f"Processing texts with spaCy for {label}"):
        doc = nlp(text)
        for token in doc:
            if token.is_alpha and token.lemma_ not in existing_words and token.lemma_ not in custom_stop_words and token.pos_ in ['ADJ', 'NOUN', 'VERB']:
                potential_words.append(token.lemma_)
    return Counter(potential_words)

# Combine both preprocessed columns
all_texts = df_scraped['translated_text_preproc'].tolist() + df_scraped['manual_summary_preproc'].tolist()

# Extract potential dovish and hawkish words using spaCy
print("Extracting potential dovish words using spaCy...")
dovish_potential_spacy = extract_potential_words(all_texts, dovish_words + hawkish_words, "dovish")

print("Extracting potential hawkish words using spaCy...")
hawkish_potential_spacy = extract_potential_words(all_texts, hawkish_words + dovish_words, "hawkish")

# Initialize TF-IDF Vectorizer with custom stop words
vectorizer = TfidfVectorizer(stop_words=custom_stop_words, max_features=1000)  # Adjust max_features as needed

# Fit and transform the text data with progress bar
print("Calculating TF-IDF scores...")
tfidf_matrix = vectorizer.fit_transform(tqdm(all_texts, desc="TF-IDF transformation"))

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame with TF-IDF scores
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Calculate mean TF-IDF scores for each word
mean_tfidf_scores = tfidf_df.mean()

# Filter potential words based on their presence in TF-IDF feature names
filtered_dovish_words = [word for word, count in dovish_potential_spacy.items() if word in feature_names]
filtered_hawkish_words = [word for word, count in hawkish_potential_spacy.items() if word in feature_names]

# Get top N TF-IDF words
top_n = 15  # Update to get top 15 words
print("Selecting top dovish words based on TF-IDF scores...")
top_dovish_tfidf = mean_tfidf_scores[filtered_dovish_words].sort_values(ascending=False).head(top_n)

print("Selecting top hawkish words based on TF-IDF scores...")
top_hawkish_tfidf = mean_tfidf_scores[filtered_hawkish_words].sort_values(ascending=False).head(top_n)

# Display the results
print("Top Dovish Words (with TF-IDF):", top_dovish_tfidf)
print("\nTop Hawkish Words (with TF-IDF):", top_hawkish_tfidf)

In [ ]:
count_df.head(30)

Now we link the sentiment analysis to each governor

In [ ]:
# Function to count sentiment words
def count_sentiment_words(text, sentiment_words):
    doc = nlp(text)
    count = 0
    for token in doc:
        if token.lemma_ in sentiment_words:
            count += 1
    return count

# Adding dovish and hawkish counts to count_df
for index, row in count_df.iterrows():
    full_name = row['Governor']
    # Filter texts containing the governor's name
    governor_texts_translated = df_scraped[df_scraped['translated_text_preproc'].str.contains(full_name, case=False, na=False)]
    governor_texts_manual = df_scraped[df_scraped['manual_summary_preproc'].str.contains(full_name, case=False, na=False)]

    # Count dovish and hawkish words in the filtered texts
    dovish_count_translated = governor_texts_translated['translated_text_preproc'].progress_apply(lambda x: count_sentiment_words(x, dovish_words)).sum()
    hawkish_count_translated = governor_texts_translated['translated_text_preproc'].progress_apply(lambda x: count_sentiment_words(x, hawkish_words)).sum()
    
    dovish_count_manual = governor_texts_manual['manual_summary_preproc'].progress_apply(lambda x: count_sentiment_words(x, dovish_words)).sum()
    hawkish_count_manual = governor_texts_manual['manual_summary_preproc'].progress_apply(lambda x: count_sentiment_words(x, hawkish_words)).sum()
    
    count_df.at[index, 'Dovish Count (Translated)'] = dovish_count_translated
    count_df.at[index, 'Hawkish Count (Translated)'] = hawkish_count_translated
    count_df.at[index, 'Dovish Count (Manual)'] = dovish_count_manual
    count_df.at[index, 'Hawkish Count (Manual)'] = hawkish_count_manual

# Calculate total sentiment-related words for each governor
count_df['Total Dovish'] = count_df['Dovish Count (Translated)'] + count_df['Dovish Count (Manual)']
count_df['Total Hawkish'] = count_df['Hawkish Count (Translated)'] + count_df['Hawkish Count (Manual)']
count_df['Total Sentiment'] = count_df['Total Dovish'] + count_df['Total Hawkish']

# Calculate the proportion of dovish and hawkish words for each governor
count_df['Dovish Proportion'] = count_df['Total Dovish'] / count_df['Total Sentiment']
count_df['Hawkish Proportion'] = count_df['Total Hawkish'] / count_df['Total Sentiment']

# Calculate the hawkish-to-dovish ratio
count_df['Hawkish/Dovish Ratio'] = count_df['Hawkish Proportion'] / count_df['Dovish Proportion']

# Normalize the hawkish and dovish proportions for comparison
count_df['Dovish Score (Normalized)'] = (count_df['Dovish Proportion'] - count_df['Dovish Proportion'].min()) / (count_df['Dovish Proportion'].max() - count_df['Dovish Proportion'].min())
count_df['Hawkish Score (Normalized)'] = (count_df['Hawkish Proportion'] - count_df['Hawkish Proportion'].min()) / (count_df['Hawkish Proportion'].max() - count_df['Hawkish Proportion'].min())

# Sort by Hawkish/Dovish Ratio in descending order
count_df_sorted = count_df.sort_values(by='Hawkish/Dovish Ratio', ascending=False)

# Display the relevant columns
print(count_df_sorted[['Governor', 'Country', 'Dovish Proportion', 'Hawkish Proportion', 'Hawkish/Dovish Ratio', 'Dovish Score (Normalized)', 'Hawkish Score (Normalized)']])

In [ ]:
count_df_sorted.head(30)

### Loughran McDonald

In [ ]:
import pandas as pd
from tqdm import tqdm
import re

# Load Loughran-McDonald Master Dictionary
file_path = 'Loughran-McDonald_MasterDictionary_1993-2023.csv'
master_dictionary_df = pd.read_csv(file_path)

# Extract sentiment words from the Loughran-McDonald dictionary
negative_words = set(master_dictionary_df[master_dictionary_df['Negative'] > 0]['Word'].str.lower())
positive_words = set(master_dictionary_df[master_dictionary_df['Positive'] > 0]['Word'].str.lower())
uncertainty_words = set(master_dictionary_df[master_dictionary_df['Uncertainty'] > 0]['Word'].str.lower())
litigious_words = set(master_dictionary_df[master_dictionary_df['Litigious'] > 0]['Word'].str.lower())
constraining_words = set(master_dictionary_df[master_dictionary_df['Constraining'] > 0]['Word'].str.lower())

# Function to count sentiment words
def count_sentiment_words(text, sentiment_words):
    words = re.findall(r'\b\w+\b', text.lower())
    count = sum(1 for word in words if word in sentiment_words)
    return count

# Adding Loughran-McDonald sentiment counts to count_df
for index, row in count_df.iterrows():
    full_name = row['Governor']
    # Filter texts containing the governor's name
    governor_texts_translated = df_scraped[df_scraped['translated_text_preproc'].str.contains(full_name, case=False, na=False)]
    governor_texts_manual = df_scraped[df_scraped['manual_summary_preproc'].str.contains(full_name, case=False, na=False)]
    
    # Concatenate all texts for each governor
    all_texts_translated = " ".join(governor_texts_translated['translated_text_preproc'])
    all_texts_manual = " ".join(governor_texts_manual['manual_summary_preproc'])
    
    # Count sentiment words in the concatenated texts
    count_df.at[index, 'LM_Negative_Count'] = count_sentiment_words(all_texts_translated, negative_words) + count_sentiment_words(all_texts_manual, negative_words)
    count_df.at[index, 'LM_Positive_Count'] = count_sentiment_words(all_texts_translated, positive_words) + count_sentiment_words(all_texts_manual, positive_words)
    count_df.at[index, 'LM_Uncertainty_Count'] = count_sentiment_words(all_texts_translated, uncertainty_words) + count_sentiment_words(all_texts_manual, uncertainty_words)
    count_df.at[index, 'LM_Litigious_Count'] = count_sentiment_words(all_texts_translated, litigious_words) + count_sentiment_words(all_texts_manual, litigious_words)
    count_df.at[index, 'LM_Constraining_Count'] = count_sentiment_words(all_texts_translated, constraining_words) + count_sentiment_words(all_texts_manual, constraining_words)

# Calculate total sentiment-related words for each governor
count_df['Total_LM_Sentiment'] = (count_df['LM_Negative_Count'] + count_df['LM_Positive_Count'] +
                                  count_df['LM_Uncertainty_Count'] + count_df['LM_Litigious_Count'] +
                                  count_df['LM_Constraining_Count'])

# Calculate the proportion of each sentiment type for each governor
count_df['LM_Positive_Proportion'] = count_df['LM_Positive_Count'] / count_df['Total_LM_Sentiment']
count_df['LM_Negative_Proportion'] = count_df['LM_Negative_Count'] / count_df['Total_LM_Sentiment']
count_df['LM_Uncertainty_Proportion'] = count_df['LM_Uncertainty_Count'] / count_df['Total_LM_Sentiment']
count_df['LM_Litigious_Proportion'] = count_df['LM_Litigious_Count'] / count_df['Total_LM_Sentiment']
count_df['LM_Constraining_Proportion'] = count_df['LM_Constraining_Count'] / count_df['Total_LM_Sentiment']

# Sort by LM Positive Proportion in descending order, then by LM Negative Proportion in ascending order
count_df_sorted = count_df.sort_values(by=['LM_Positive_Proportion', 'LM_Negative_Proportion'], ascending=[False, True])

# Display the relevant columns
print(count_df_sorted[['Governor', 'Country', 'LM_Positive_Count', 'LM_Negative_Count', 'LM_Uncertainty_Count', 'LM_Litigious_Count', 'LM_Constraining_Count', 'LM_Positive_Proportion', 'LM_Negative_Proportion', 'LM_Uncertainty_Proportion', 'LM_Litigious_Proportion', 'LM_Constraining_Proportion']])


In [ ]:
count_df_sorted.head(30)[['Governor', 'Country', 'LM_Positive_Count', 'LM_Negative_Count', 'LM_Uncertainty_Count', 'LM_Litigious_Count', 'LM_Constraining_Count', 'LM_Positive_Proportion', 'LM_Negative_Proportion', 'LM_Uncertainty_Proportion', 'LM_Litigious_Proportion', 'LM_Constraining_Proportion']]
